### After understanding the data and seeing the results of simple thresholding, it is imperative to use some more refined methods to detect the spikes. Simple thresholding counts some extra peaks.

Simple thresholding would have worked in case the pulses were in only one direction, i.e. in the negative direction as is in the usual brain recordings. This data seems to have pulses in both the directions. This makes things a little more complicated. In this script, I will try to implement an algorithm to detect the spikes by tracing the neural spike through it's stages of hypopolarization, depolarization, overshoot, repolarization and hyperpolarization.

In [1]:
import numpy as np
import pylab as pl

In [2]:
output = np.loadtxt('./data/output5.dat');

In [3]:
test_samples = output[1300:1400]
template = output[1330:1355]
sigma = 22
N = 6

hypo = False
depo = False
over = False
repo = False
hype = True
local_min = 5000
local_max = -5000
samples_without_fire = 0
bw = 30

for i in range(len(test_samples)):
    print(hypo, depo, over, repo, hype, i)
    print(local_min, local_max, test_samples[i])
    if samples_without_fire > bw and hypo == True:
        samples_without_fire = 0
        hype = True
        depo = over = repo = hypo = False
        local_max = samples[i]
        local_min = samples[i]
    if test_samples[i]<local_min:
        local_min = test_samples[i]
        if hype and abs(test_samples[i]-local_max)<2*sigma:
            hypo = True
            depo = over = repo = hype = False
            samples_without_fire = 0
    if test_samples[i]>local_max:
        local_max = test_samples[i]
    if hypo and not depo and not over and not repo and not hype:
        if test_samples[i]>local_min:
            local_max = test_samples[i]
            hypo = False
            depo = True
    if not hypo and depo and not over and not repo and not hype:
        if test_samples[i]<local_max:
            depo = False
            over = True
            repo = True
            local_min = test_samples[i]
    if not hypo and not depo and over and repo and not hype:
        if test_samples[i]>local_min:
            hype = True
            over = repo = False
    if (hype == True) and (local_max-local_min > N * sigma) and (local_max>0) and (local_min<0):
        print("spike up occured at", i)
        local_max = test_samples[i]
        local_min = test_samples[i]
        hype = True
        depo = over = repo = hypo = False
    elif hype:
        local_max = test_samples[i]
        local_min = test_samples[i]
        hype = True
        depo = over = repo = hypo = False
    samples_without_fire += 1

False False False False True 0
5000 -5000 -31.0
False False False False True 1
-31.0 -31.0 -29.0
False False False False True 2
-29.0 -29.0 -25.0
False False False False True 3
-25.0 -25.0 -21.0
False False False False True 4
-21.0 -21.0 -17.0
False False False False True 5
-17.0 -17.0 -15.0
False False False False True 6
-15.0 -15.0 -14.0
False False False False True 7
-14.0 -14.0 -16.0
True False False False False 8
-16.0 -14.0 -20.0
True False False False False 9
-20.0 -14.0 -28.0
True False False False False 10
-28.0 -14.0 -36.0
True False False False False 11
-36.0 -14.0 -39.0
True False False False False 12
-39.0 -14.0 -39.0
True False False False False 13
-39.0 -14.0 -36.0
False True False False False 14
-39.0 -36.0 -31.0
False True False False False 15
-39.0 -31.0 -26.0
False True False False False 16
-39.0 -26.0 -22.0
False True False False False 17
-39.0 -22.0 -20.0
False True False False False 18
-39.0 -20.0 -19.0
False True False False False 19
-39.0 -19.0 -19.0
False True 

This doesn't feel so refined and has too many variables to take care of. I need to find a simpler way of doing this. Since this is only taking the directions into account, maybe I don't need to tract all the positions, and I can simply take care of peaks and note when direction changes. Keep it in an array. Then check if that array has a spike like appearance and if so, check it for some threshold of spiking activity reached. If so, it will definitely be a spike.